In [1]:
import pandas as pd
import string
import nltk
nltk.download('stopwords')
from collections import Counter
from nltk.corpus import stopwords
import math
import numpy as np
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/aaditya11/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def docPreProcessing(filepath):
    stopwords_dict = Counter(stopwords.words('english'))

    file = open(filepath, 'r')
    lines = file.readlines()
    
    print(filepath)

    stopwords_dict = Counter(stopwords.words('english'))
    
    
    doctext = ""
    for line in lines:
        line = line.translate(str.maketrans('', '', string.punctuation)).strip().lower()
        line = ' '.join([word for word in line.split() if word not in stopwords_dict])
        doctext += line
        
    doctextList = doctext.split()
    return doctextList

In [33]:
def tf(wordlist):
    # global tfcount
    # tfcount += 1
    wordmap = {}
    
    for word in wordlist:
        if word in wordmap:
            wordmap[word] += 1
        else:
            wordmap[word] = 1
            
    
    numWords = len(wordlist)
    
    for word in wordmap:
        wordmap[word] = wordmap[word] / numWords
    
    return wordmap
        


In [34]:
def tfidfMapBuilder(doclist):
    termset = set()

    for doc in doclist:
        termset.update(set(docPreProcessing(doc)))

    df_columns = doclist.copy()
    df_columns.append('df')
    print(df_columns)

    df = pd.DataFrame(0, index=list(termset), columns=df_columns)

    for doc in doclist:
        wordmap = tf(docPreProcessing(doc))
        for term in wordmap:
            df.at[term, doc] = wordmap[term]
            df.at[term, 'df'] += 1

    # print(df)

    return df

    # print(len(termset))

In [35]:
def fileCollector(path):
    filelist = []
    
    for (root, dirs, files) in os.walk(path, topdown=True):
        for file in files:
            filelist.append(os.path.join(root, file))
            
            # if len(filelist) == 20:
            #     return filelist

    return filelist

In [36]:
doclist = []

# doclist.append('query.txt')

doclist.extend(fileCollector(r'../testset/sci.space'))

tfidfMapBuilder(doclist).to_csv('tfidf.csv')
# tfidfMapBuilder.to_csv('tfidf.csv')



../testset/sci.space/60777
../testset/sci.space/61312
../testset/sci.space/61023
../testset/sci.space/61090
../testset/sci.space/61145
../testset/sci.space/61245
../testset/sci.space/61042
../testset/sci.space/61433
../testset/sci.space/61064
../testset/sci.space/60978
../testset/sci.space/60894
../testset/sci.space/60910
../testset/sci.space/60785
../testset/sci.space/60252
../testset/sci.space/61035
../testset/sci.space/60888
../testset/sci.space/61186
../testset/sci.space/60253
../testset/sci.space/61138
../testset/sci.space/62477
['../testset/sci.space/60777', '../testset/sci.space/61312', '../testset/sci.space/61023', '../testset/sci.space/61090', '../testset/sci.space/61145', '../testset/sci.space/61245', '../testset/sci.space/61042', '../testset/sci.space/61433', '../testset/sci.space/61064', '../testset/sci.space/60978', '../testset/sci.space/60894', '../testset/sci.space/60910', '../testset/sci.space/60785', '../testset/sci.space/60252', '../testset/sci.space/61035', '../tests